In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
!cd /content/

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Check for GPU
!nvidia-smi

# Install NeMo (or other Tacotron 2 implementation)
!pip install nemo_toolkit['all']
!pip install datasets

import nemo
import os

Wed Apr 23 11:19:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8             14W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!rm -rf /content/NeMo  # Remove the old NeMo directory (if it exists)
%cd /content/
!git clone https://github.com/NVIDIA/NeMo.git
!ls

/content
Cloning into 'NeMo'...
remote: Enumerating objects: 232469, done.
remote: Counting objects: 100% (2454/2454), done.
remote: Compressing objects: 100% (947/947), done.
remote: Total 232469 (delta 2125), reused 1509 (delta 1507), pack-reused 230015 (from 3)
Receiving objects: 100% (232469/232469), 441.56 MiB | 33.69 MiB/s, done.
Resolving deltas: 100% (175335/175335), done.
NeMo  sample_data


In [ ]:
from datasets import load_dataset
import soundfile as sf
import torch

# Load the Slovenian split of the Common Voice 16.0 dataset
common_voice_sl = load_dataset("mozilla-foundation/common_voice_16_0", "sl")

print(common_voice_sl)

# Example of accessing an audio sample and its transcription
example = common_voice_sl["train"][0]
print(example)
print(f"Transcription: {example['sentence']}")
print(f"Audio: {example['audio']}")

# You can listen to the audio (this might require specific libraries in Colab)
# import IPython.display as ipd
# ipd.Audio(data=example['audio']['array'], rate=example['audio']['sampling_rate'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

common_voice_16_0.py:   0%|          | 0.00/8.17k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/77.3k [00:00<?, ?B/s]

...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 3228it [00:00, 88400.88it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 278it [00:00, 64760.71it/s]


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 1404
    })
    validation: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 1211
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 1238
    })
    other: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 3228
    })
    invalidated: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows

In [ ]:
import os
import json

%cd /content/

OUTPUT_DIR = "/content/common_voice_audio"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def create_nemo_manifest(dataset_split, output_filename):
    manifest = []
    for i, example in enumerate(dataset_split):
        audio_data = example["audio"]["array"]
        sampling_rate = example["audio"]["sampling_rate"]
        transcription = example["sentence"].strip()

        audio_filename = f"{dataset_split.split}-{i}.wav"
        audio_filepath = os.path.join(OUTPUT_DIR, audio_filename)

        # Save the audio to a WAV file
        sf.write(audio_filepath, audio_data, sampling_rate)

        # Create the manifest entry with only the available fields
        manifest_entry = {
            "audio_filepath": audio_filepath,
            "text": transcription,
        }
        manifest.append(manifest_entry)

    with open(output_filename, "w", encoding="utf-8", newline="\n") as f:
      for entry in manifest:
          json_string = json.dumps(entry, ensure_ascii=False)
          print(f"Writing line: {json_string}")  # Print the line
          f.write(json_string + "\n")

# Create manifest files for training and validation splits (if available)
if "train" in common_voice_sl:
    create_nemo_manifest(common_voice_sl["train"], "train_manifest.json")
    print("Training manifest created: train_manifest.json")

if "validation" in common_voice_sl:
    create_nemo_manifest(common_voice_sl["validation"], "val_manifest.json")
    print("Validation manifest created: val_manifest.json")
elif "test" in common_voice_sl:
    create_nemo_manifest(common_voice_sl["test"], "val_manifest.json") # Using test as validation if no validation split
    print("Test manifest created and used as validation: val_manifest.json")
else:
    print("No validation or test split found. You might need to create one.")

/content
Writing line: {"audio_filepath": "/content/common_voice_audio/train-0.wav", "text": "Oh, sedaj si pa stvari že izmišljuješ."}
Writing line: {"audio_filepath": "/content/common_voice_audio/train-1.wav", "text": "Kaj pa vem, zakaj pa ne?"}
Writing line: {"audio_filepath": "/content/common_voice_audio/train-2.wav", "text": "Kaj je to, Tomaž?"}
Writing line: {"audio_filepath": "/content/common_voice_audio/train-3.wav", "text": "Popravila je tvojo enačbo."}
Writing line: {"audio_filepath": "/content/common_voice_audio/train-4.wav", "text": "Prepričana sem, da obstaja nekje punca ravno pravšnja zate."}
Writing line: {"audio_filepath": "/content/common_voice_audio/train-5.wav", "text": "Me sploh poslušaš?"}
Writing line: {"audio_filepath": "/content/common_voice_audio/train-6.wav", "text": "Skoči čez ograjo in pohiti v banko."}
Writing line: {"audio_filepath": "/content/common_voice_audio/train-7.wav", "text": "En dan za drugim."}
Writing line: {"audio_filepath": "/content/common_voi

Save this yaml file in tacotron2_my_config.yaml.

```yaml
name: Tacotron2-CommonVoiceSL  # Changed name

train_dataset: "/content/train_manifest.json"  # Adjusted path
validation_datasets: "/content/val_manifest.json"  # Adjusted path
sup_data_path: null
sup_data_types: null

phoneme_dict_path: "scripts/tts_dataset_files/cmudict-0.7b_nv22.10"
heteronyms_path: "scripts/tts_dataset_files/heteronyms-052722"

model:
  pitch_fmin: 65.40639132514966
  pitch_fmax: 2093.004522404789

  sample_rate: 16000  # Changed to 16kHz
  n_mel_channels: 80
  n_window_size: 1024
  n_window_stride: 256
  n_fft: 1024
  lowfreq: 0
  highfreq: 7600  # Adjusted highfreq
  window: hann
  pad_value: -11.52

  # text_normalizer:
  #   _target_: nemo_text_processing.text_normalization.normalize.Normalizer
  #   lang: sl  # Changed to Slovenian
  #   input_case: cased

  # text_normalizer_call_kwargs:
  #   verbose: false
  #   punct_pre_process: true
  #   punct_post_process: true

  text_tokenizer:
    _target_: nemo.collections.common.tokenizers.text_to_speech.tts_tokenizers.EnglishPhonemesTokenizer
    punct: true
    stresses: false
    chars: true
    apostrophe: true
    pad_with_space: true
    g2p:
      _target_: nemo.collections.tts.g2p.models.en_us_arpabet.EnglishG2p  # Keep English G2P for now
      phoneme_dict: ${phoneme_dict_path}
      heteronyms: ${heteronyms_path}

  train_ds:
    dataset:
      _target_: "nemo.collections.tts.data.dataset.TTSDataset"
      manifest_filepath: "/content/train_manifest.json"
      sample_rate: 16000
      sup_data_path: null
      sup_data_types: null
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 7600
      max_duration: null
      min_duration: 0.1
      ignore_file: null
      trim: False
      pitch_fmin: 65.40639132514966
      pitch_fmax: 2093.004522404789
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 32  # Adjusted batch size
      num_workers: 4
      pin_memory: True

  validation_ds:
    dataset:
      _target_: "nemo.collections.tts.data.dataset.TTSDataset"
      manifest_filepath: "/content/val_manifest.json"  # Direct path
      sample_rate: 16000  # Direct value
      sup_data_path: null
      sup_data_types: null
      n_fft: 1024  # Direct value
      win_length: 1024  # Direct value
      hop_length: 256  # Direct value
      window: hann  # Direct value
      n_mels: 80  # Direct value
      lowfreq: 0  # Direct value
      highfreq: 7600  # Direct value
      max_duration: null
      min_duration: 0.1
      ignore_file: null
      trim: False
      pitch_fmin: 65.40639132514966  # Direct value
      pitch_fmax: 2093.004522404789  # Direct value
    dataloader_params:
      drop_last: false
      shuffle: false
      batch_size: 16  # Adjusted batch size
      num_workers: 4
      pin_memory: True

  preprocessor:
    _target_: nemo.collections.asr.parts.preprocessing.features.FilterbankFeatures
    nfilt: ${model.n_mel_channels}
    highfreq: ${model.highfreq}
    log: True
    log_zero_guard_type: clamp
    log_zero_guard_value: 1e-05
    lowfreq: ${model.lowfreq}
    n_fft: ${model.n_fft}
    n_window_size: ${model.n_window_size}
    n_window_stride: ${model.n_window_stride}
    pad_to: 16
    pad_value: ${model.pad_value}
    sample_rate: ${model.sample_rate}
    window: ${model.window}
    normalize: null
    preemph: null
    dither: 0.0
    frame_splicing: 1
    stft_conv: False
    nb_augmentation_prob: 0
    mag_power: 1.0
    exact_pad: True
    use_grads: False

  encoder:
    _target_: nemo.collections.tts.modules.tacotron2.Encoder
    encoder_kernel_size: 5
    encoder_n_convolutions: 3
    encoder_embedding_dim: 512

  decoder:
    _target_: nemo.collections.tts.modules.tacotron2.Decoder
    decoder_rnn_dim: 1024
    encoder_embedding_dim: ${model.encoder.encoder_embedding_dim}
    gate_threshold: 0.5
    max_decoder_steps: 1000
    n_frames_per_step: 1  # currently only 1 is supported
    n_mel_channels: ${model.n_mel_channels}
    p_attention_dropout: 0.1
    p_decoder_dropout: 0.1
    prenet_dim: 256
    prenet_p_dropout: 0.5
    # Attention parameters
    attention_dim: 128
    attention_rnn_dim: 1024
    # AttentionLocation Layer parameters
    attention_location_kernel_size: 31
    attention_location_n_filters: 32
    early_stopping: True

  postnet:
    _target_: nemo.collections.tts.modules.tacotron2.Postnet
    n_mel_channels: ${model.n_mel_channels}
    p_dropout: 0.5
    postnet_embedding_dim: 512
    postnet_kernel_size: 5
    postnet_n_convolutions: 5

  optim:
    name: adam
    lr: 1e-3
    weight_decay: 1e-6

    # scheduler setup
    sched:
      name: CosineAnnealing
      min_lr: 1e-5

trainer:
  devices: 1  # number of gpus
  max_epochs: 100
  num_nodes: 1
  accelerator: gpu
  strategy: auto  # Changed from ddp
  accumulate_grad_batches: 1
  enable_checkpointing: True  # Changed from False
  logger: False
  gradient_clip_val: 1.0
  log_every_n_steps: 60
  check_val_every_n_epoch: 1  # Adjusted
  benchmark: False
  callbacks: []

exp_manager:
  exp_dir: null
  name: ${name}
  create_tensorboard_logger: True
  create_checkpoint_callback: False
  checkpoint_callback_params:
    monitor: val_loss
    mode: min
```

In [ ]:
import yaml

config = {
    "name": "Tacotron2-CommonVoiceSL",  # Changed name
    "train_dataset": "/content/train_manifest.json",  # Adjusted path
    "validation_datasets": "/content/val_manifest.json",  # Adjusted path
    "sup_data_path": None,
    "sup_data_types": None,
    "phoneme_dict_path": "scripts/tts_dataset_files/cmudict-0.7b_nv22.10",
    "heteronyms_path": "scripts/tts_dataset_files/heteronyms-052722",
    "model": {
        "pitch_fmin": 65.40639132514966,
        "pitch_fmax": 2093.004522404789,
        "sample_rate": 16000,  # Changed to 16kHz
        "n_mel_channels": 80,
        "n_window_size": 1024,
        "n_window_stride": 256,
        "n_fft": 1024,
        "lowfreq": 0,
        "highfreq": 7600,  # Adjusted highfreq
        "window": "hann",
        "pad_value": -11.52,
        "text_tokenizer": {
            "_target_": "nemo.collections.common.tokenizers.text_to_speech.tts_tokenizers.BaseCharsTokenizer",
            "punct": True,
            "pad_with_space": True,
            "chars": ["a", "b", "c", "č", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "r", "s", "š", "t", "u", "v", "z", "ž",
                      "A", "B", "C", "Č", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "R", "S", "Š", "T", "U", "V", "Z", "Ž",
                      " ", "'", "-", ",", ".", "!", "?"]
        },
        "train_ds": {
            "dataset": {
                "_target_": "nemo.collections.tts.data.dataset.TTSDataset",
                "manifest_filepath": "/content/train_manifest.json",
                "sample_rate": 16000,
                "sup_data_path": None,
                "sup_data_types": None,
                "n_fft": 1024,
                "win_length": 1024,
                "hop_length": 256,
                "window": "hann",
                "n_mels": 80,
                "lowfreq": 0,
                "highfreq": 7600,
                "max_duration": None,
                "min_duration": 0.1,
                "ignore_file": None,
                "trim": False,
                "pitch_fmin": 65.40639132514966,
                "pitch_fmax": 2093.004522404789
            },
            "dataloader_params": {
                "drop_last": False,
                "shuffle": True,
                "batch_size": 32,  # Adjusted batch size
                "num_workers": 4,
                "pin_memory": True
            }
        },
        "validation_ds": {
            "dataset": {
                "_target_": "nemo.collections.tts.data.dataset.TTSDataset",
                "manifest_filepath": "/content/val_manifest.json",  # Direct path
                "sample_rate": 16000,  # Direct value
                "sup_data_path": None,
                "sup_data_types": None,
                "n_fft": 1024,  # Direct value
                "win_length": 1024,  # Direct value
                "hop_length": 256,  # Direct value
                "window": "hann",  # Direct value
                "n_mels": 80,  # Direct value
                "lowfreq": 0,  # Direct value
                "highfreq": 7600,  # Direct value
                "max_duration": None,
                "min_duration": 0.1,
                "ignore_file": None,
                "trim": False,
                "pitch_fmin": 65.40639132514966,  # Direct value
                "pitch_fmax": 2093.004522404789  # Direct value
            },
            "dataloader_params": {
                "drop_last": False,
                "shuffle": False,
                "batch_size": 16,  # Adjusted batch size
                "num_workers": 4,
                "pin_memory": True
            }
        },
        "preprocessor": {
            "_target_": "nemo.collections.asr.parts.preprocessing.features.FilterbankFeatures",
            "nfilt": "${model.n_mel_channels}",
            "highfreq": "${model.highfreq}",
            "log": True,
            "log_zero_guard_type": "clamp",
            "log_zero_guard_value": 1e-05,
            "lowfreq": "${model.lowfreq}",
            "n_fft": "${model.n_fft}",
            "n_window_size": "${model.n_window_size}",
            "n_window_stride": "${model.n_window_stride}",
            "pad_to": 16,
            "pad_value": "${model.pad_value}",
            "sample_rate": "${model.sample_rate}",
            "window": "${model.window}",
            "normalize": None,
            "preemph": None,
            "dither": 0.0,
            "frame_splicing": 1,
            "stft_conv": False,
            "nb_augmentation_prob": 0,
            "mag_power": 1.0,
            "exact_pad": True,
            "use_grads": False
        },
        "encoder": {
            "_target_": "nemo.collections.tts.modules.tacotron2.Encoder",
            "encoder_kernel_size": 5,
            "encoder_n_convolutions": 3,
            "encoder_embedding_dim": 512
        },
        "decoder": {
            "_target_": "nemo.collections.tts.modules.tacotron2.Decoder",
            "decoder_rnn_dim": 1024,
            "encoder_embedding_dim": "${model.encoder.encoder_embedding_dim}",
            "gate_threshold": 0.5,
            "max_decoder_steps": 1000,
            "n_frames_per_step": 1,  # currently only 1 is supported
            "n_mel_channels": "${model.n_mel_channels}",
            "p_attention_dropout": 0.1,
            "p_decoder_dropout": 0.1,
            "prenet_dim": 256,
            "prenet_p_dropout": 0.5,
            # Attention parameters
            "attention_dim": 128,
            "attention_rnn_dim": 1024,
            # AttentionLocation Layer parameters
            "attention_location_kernel_size": 31,
            "attention_location_n_filters": 32,
            "early_stopping": True
        },
        "postnet": {
            "_target_": "nemo.collections.tts.modules.tacotron2.Postnet",
            "n_mel_channels": "${model.n_mel_channels}",
            "p_dropout": 0.5,
            "postnet_embedding_dim": 512,
            "postnet_kernel_size": 5,
            "postnet_n_convolutions": 5
        },
        "loss": {
            "type": "l1",
            "params": {}
        },
        "optim": {
            "name": "adam",
            "lr": 1e-3,
            "weight_decay": 1e-6,
            # scheduler setup
            "sched": {
                "name": "CosineAnnealing",
                "min_lr": 1e-5
            }
        }
    },
    "trainer": {
        "devices": 1,  # number of gpus
        "max_epochs": 100,
        "num_nodes": 1,
        "accelerator": "gpu",
        "strategy": "auto",  # Changed from ddp
        "accumulate_grad_batches": 1,
        "enable_checkpointing": True,  # Changed from False
        "logger": False,
        "gradient_clip_val": 1.0,
        "log_every_n_steps": 60,
        "check_val_every_n_epoch": 1,  # Adjusted
        "benchmark": False,
        "callbacks": []
    },
    "exp_manager": {
        "exp_dir": None,
        "name": "${name}",
        "create_tensorboard_logger": True,
        "create_checkpoint_callback": False,
        "checkpoint_callback_params": {
            "monitor": "val_loss",
            "mode": "min"
        }
    }
}

with open("/content/tacotron2_my_config.yaml", "w") as f:
    yaml.dump(config, f, indent=4)

print("Configuration file created: /content/tacotron2_my_config.yaml")

Configuration file created: /content/tacotron2_my_config.yaml


In [ ]:
import json

%cd /content/

try:
    with open("/content/train_manifest.json", "r", encoding="utf-8") as f:
        for raw_line in f:
            line = raw_line.strip()  # Remove leading/trailing whitespace
            if line:  # Skip empty lines
                try:
                    item = json.loads(line)
                except json.JSONDecodeError as e:
                    print("Error parsing line:", line)
                    print("Error:", e)
except FileNotFoundError:
    print("Error: File not found - /content/train_manifest.json")

/content


In [ ]:
import os
abs_config_path = os.path.abspath("/content/tacotron2_my_config.yaml")  # Get the absolute path
print(f"Absolute path to config: {abs_config_path}")

%cd /content/
%cp tacotron2_my_config.yaml /content/NeMo/examples/tts/conf/
%cd /content/NeMo/
!python examples/tts/tacotron2.py --config-path=/content/NeMo/examples/tts/conf/ --config-name=tacotron2_my_config

Streaming output truncated to the last 5000 lines.
[NeMo I 2025-04-23 11:28:31 nemo_logging:393] Not all audio files have duration information. Duration logging will be disabled.
[NeMo I 2025-04-23 11:28:31 nemo_logging:393] Not all audio files have duration information. Duration logging will be disabled.
[NeMo I 2025-04-23 11:28:31 nemo_logging:393] Not all audio files have duration information. Duration logging will be disabled.
[NeMo I 2025-04-23 11:28:31 nemo_logging:393] Not all audio files have duration information. Duration logging will be disabled.
[NeMo I 2025-04-23 11:28:31 nemo_logging:393] Not all audio files have duration information. Duration logging will be disabled.
[NeMo I 2025-04-23 11:28:31 nemo_logging:393] Not all audio files have duration information. Duration logging will be disabled.
[NeMo I 2025-04-23 11:28:31 nemo_logging:393] Not all audio files have duration information. Duration logging will be disabled.
[NeMo I 2025-04-23 11:28:31 nemo_logging:393] Not all